In [1]:
import pandas as pd
import torch
import numpy as np
from scipy.stats import spearmanr
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [38]:
log = pd.read_pickle("/home/dhvc/projetos/Grad-Eclip/results")

log = {
        'img_id': [],
        'interpretability_method': [],
        'label': [],
        'similarity_original': [],
        'similarity_perturb': [],
        'hm_original': [],
        'hm_perturbed': [],
        'alpha': [],
        'clip_model': []
    }

In [39]:
log.size

198045

In [40]:
log.head()

,img_id,interpretability_method,label,similarity_original,similarity_perturb,hm_original,hm_perturbed,alpha,clip_model
0,0,selfattn,9,0.297363,0.306641,"[[tensor(0., dtype=torch.float16), tensor(0., ...","[[tensor(0., dtype=torch.float16), tensor(0., ...",0.062745,ViT-B/16
1,0,gradcam,9,0.297363,0.306641,"[[tensor(0.2167, dtype=torch.float16), tensor(...","[[tensor(0.2167, dtype=torch.float16), tensor(...",0.062745,ViT-B/16
2,0,maskclip,9,0.297363,0.306641,"[[tensor(0.1458, dtype=torch.float16), tensor(...","[[tensor(0.1458, dtype=torch.float16), tensor(...",0.062745,ViT-B/16
3,0,eclip,9,0.297363,0.306641,"[[tensor(0.0619, dtype=torch.float16), tensor(...","[[tensor(0.0619, dtype=torch.float16), tensor(...",0.062745,ViT-B/16
4,0,game,9,0.297363,0.306641,"[[tensor(0.0138, dtype=torch.float16), tensor(...","[[tensor(0.0469, dtype=torch.float16), tensor(...",0.062745,ViT-B/16


In [57]:
group_name = "gradcam"
filtered_group = log[log['interpretability_method'] == group_name].head()
filtered_group

,img_id,interpretability_method,label,similarity_original,similarity_perturb,hm_original,hm_perturbed,alpha,clip_model
1,0,gradcam,9,0.297363,0.306641,"[[tensor(0.2167, dtype=torch.float16), tensor(...","[[tensor(0.2167, dtype=torch.float16), tensor(...",0.062745,ViT-B/16
10,1,gradcam,9,0.284180,0.302246,"[[tensor(0.2122, dtype=torch.float16), tensor(...","[[tensor(0.2122, dtype=torch.float16), tensor(...",0.062745,ViT-B/16
19,2,gradcam,9,0.259277,0.283691,"[[tensor(0.1674, dtype=torch.float16), tensor(...","[[tensor(0.1674, dtype=torch.float16), tensor(...",0.062745,ViT-B/16
28,3,gradcam,9,0.269531,0.281738,"[[tensor(0.1893, dtype=torch.float16), tensor(...","[[tensor(0.1893, dtype=torch.float16), tensor(...",0.062745,ViT-B/16
37,4,gradcam,9,0.257324,0.297852,"[[tensor(0.2197, dtype=torch.float16), tensor(...","[[tensor(0.2197, dtype=torch.float16), tensor(...",0.062745,ViT-B/16


In [58]:
hm_original_values = []
hm_perturbed_values = []

for item in log[log['interpretability_method'] == group_name].head().iterrows():
    for tensor in item[1]['hm_original']:
        hm_original_values.extend(tensor.flatten().cpu().numpy())      
    for tensor in item[1]['hm_perturbed']:
        hm_perturbed_values.extend(tensor.flatten().cpu().numpy()) 
        
    spearman_corr, _ = spearmanr(hm_original_values, hm_perturbed_values)
    print(f"Spearman rank correlation for {methd}: {spearman_corr}")

Spearman rank correlation for rise: 1.0
Spearman rank correlation for rise: 1.0
Spearman rank correlation for rise: 1.0
Spearman rank correlation for rise: 0.9999999999999999
Spearman rank correlation for rise: 0.9999999999999998


In [36]:
def calculate_spearman_for_group(group):
    hm_original_values = []
    hm_perturbed_values = []

    
    for i, row in group.iterrows():
        for tensor in row['hm_original']:
            hm_original_values.extend(tensor.flatten().cpu().numpy())  
        
        for tensor in row['hm_perturbed']:
            hm_perturbed_values.extend(tensor.flatten().cpu().numpy())  

    spearman_corr, _ = spearmanr(hm_original_values, hm_perturbed_values)
    return spearman_corr

int_method_list =  ["selfattn", "gradcam", "maskclip", "eclip", "game", "rollout", "surgery", "m2ib", "rise"]

for methd in int_method_list:
    filtered_group = log[log['interpretability_method'] == methd]
    spearman_corr = calculate_spearman_for_group(filtered_group)
    print(f"Spearman rank correlation for {methd}: {spearman_corr}")


Spearman rank correlation for selfattn: 1.0
Spearman rank correlation for gradcam: 1.0
Spearman rank correlation for maskclip: 0.9999999999999998
Spearman rank correlation for eclip: 0.9999999999999998
Spearman rank correlation for game: 0.4062111851129004
Spearman rank correlation for rollout: 0.4648405285133196
Spearman rank correlation for surgery: 0.6993520916851954
Spearman rank correlation for m2ib: 0.44047865253871205
Spearman rank correlation for rise: 0.0037022300144166492


In [12]:
def topk_intersection(map1, map2, k):
    flat1 = map1.flatten()
    flat2 = map2.flatten()
    
    topk_indices_1 = torch.topk(flat1.clone().detach(), k).indices
    topk_indices_2 = torch.topk(flat2.clone().detach(), k).indices

    set1 = set(topk_indices_1.tolist())
    set2 = set(topk_indices_2.tolist())    
    
    intersection = set1.intersection(set2)
    
    return len(intersection), len(intersection) / k

# Valor de k (pode ajustar)
k = 100

for idx, row in filtered_data.iterrows():
    # Converter para tensores
    map1 = torch.tensor([[t.item() for t in inner] for inner in row["hm_original"]])
    map2 = torch.tensor([[t.item() for t in inner] for inner in row["hm_perturbed"]])
    
    # Calcular interseção
    inter_count, inter_ratio = topk_intersection(map1, map2, k)
    print(f"Top-{k} intersection count: {inter_count}")
    print(f"Top-{k} intersection ratio: {inter_ratio:.3f}") 

Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
Top-100 intersection count: 100
Top-100 intersection ratio: 1.000
